In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas
import src.library.graph_functions as graph_functions
from sklearn.neighbors import KDTree
import pickle
import networkx
import random
import math
import geopandas as gpd
from config.config import DATA_DIR


In [3]:
path = os.getcwd()
parent = os.path.abspath(os.path.join(path, os.pardir))
data_directory = DATA_DIR
results_directory = parent+'/results'

print(data_directory)
print(results_directory)

c:\users\joans\onedrive\escriptori\icra\wotter\data
c:\Users\joans\OneDrive\Escriptori\icra\wOtter/results


In [4]:
#Read AGG_WWTP file (contains WWTP and discharge points)
contamination_df_location = os.path.join(data_directory, "AGG_WWTP_df.csv")
cont_df = pandas.read_csv(contamination_df_location)
cont_df_names = cont_df[(cont_df.dcpLatitud > 0) | (cont_df.dcpLongitu > 0)]
cont_df_names = cont_df_names.dropna(subset=['dcpLatitud', 'dcpLongitu'])
cont_df_names

,Unnamed: 0,pixel_number,Treatment_level,Treat_a,Filt_a,Unfilt_a,pollution,dcpLatitud,dcpLongitu,countryID
0,0,95887462,2,2377.613912,0.000000,0.000000,1,47.707908,17.095127,1
1,1,96282634,2,9439.710173,0.000000,0.000000,1,47.594998,16.640054,1
2,2,97995469,2,1170.979096,0.000000,0.000000,1,47.110822,16.454944,1
3,3,95448129,2,18737.363434,0.000000,0.000000,1,47.830604,16.541495,1
4,4,95477502,2,2213.484411,0.000000,0.000000,1,47.826894,16.922604,1
...,...,...,...,...,...,...,...,...,...,...
49957,49957,74713695,0,0.000000,200.625681,70.511418,1,53.735417,-0.939583,31
49958,49958,77846924,0,0.000000,518.645337,182.281840,1,52.837437,0.175265,31
49959,49959,82897559,0,0.000000,0.000000,0.000000,1,51.400446,-0.508509,31
49960,49960,72911320,0,0.000000,13.664907,4.802635,1,54.246337,-7.853090,31


In [5]:
#Read UWWTPS file (contains name of WWTP)
wwtp_location = os.path.join(data_directory, "UWWTPS.csv")   #Data from https://www.eea.europa.eu/data-and-maps/data/waterbase-uwwtd-urban-waste-water-treatment-directive-8
wwtp_df = pandas.read_csv(wwtp_location)
wwtp_df = wwtp_df[['uwwLatitude', 'uwwLongitude', 'uwwName']]
wwtp_df = wwtp_df.dropna(subset=['uwwLatitude', 'uwwLongitude'])
wwtp_df

,uwwLatitude,uwwLongitude,uwwName
0,50.757100,5.584600,BASSENGE
1,50.636600,4.540600,BOUSVAL
2,50.567600,5.629300,CHAUDFONTAINE
3,50.476800,5.588500,COMBLAIN-AU-PONT
4,50.476200,5.578200,COMBLAIN-AU-PONT
...,...,...,...
28570,54.124470,-8.304280,Ballysadare Waste Water Treatment Plant
28571,54.186304,-8.486182,Collooney Waste Water Treatment Plant
28572,52.586303,-6.488664,Ferns Waste Water Treatment Plant
28573,52.854403,-6.320819,Aughrim Waste Water Treatment Plant


In [6]:
#Merge UWWTPS and AGG_WTP by coordinates
kd = KDTree(wwtp_df[["uwwLatitude", "uwwLongitude"]].values, metric='euclidean')
distances, indices = kd.query(cont_df_names[["dcpLatitud", "dcpLongitu"]], k = 1)
indices = map(lambda index: index[0], indices)
cont_df_names['uwwName'] = list(wwtp_df.iloc[indices, 2])

cont_df_names["Population"] = cont_df_names["Treat_a"] + cont_df_names["Filt_a"] + cont_df_names["Unfilt_a"]
cont_df_names = cont_df_names[cont_df_names["Population"] > 0]  #If it does not treat population, it's not a WWTP

cont_df_names = cont_df_names[['uwwName', 'Population', 'Treatment_level', 'dcpLongitu', 'dcpLatitud']]
cont_df_names["Population (thousands of inhabitants)"] = cont_df_names["Population"].div(1000).round(4)

cont_df_names

,uwwName,Population,Treatment_level,dcpLongitu,dcpLatitud,Population (thousands of inhabitants)
0,Andau,2377.613912,2,17.095127,47.707908,2.3776
1,Deutschkreuz (Mittleres Burgenland) Goldbachtal,9439.710173,2,16.640054,47.594998,9.4397
2,Deutsch Schützen (Deutsch Schützen - Höll),1170.979096,2,16.454944,47.110822,1.1710
3,Eisenstadt (Eisbachtal),18737.363434,2,16.541495,47.830604,18.7374
4,Frauenkirchen,2213.484411,2,16.922604,47.826894,2.2135
...,...,...,...,...,...,...
49956,STRATHPEFFER WWTP,631.254177,0,-4.597897,58.067219,0.6313
49957,GOOLE STW,271.137100,0,-0.939583,53.735417,0.2711
49958,SUTTON BRIDGE STW,700.927178,0,0.175265,52.837437,0.7009
49960,Enniskillen,18.467542,0,-7.853090,54.246337,0.0185


In [7]:
#Read observations file
observations = os.path.join(data_directory, "observations.csv")
observations_df = pandas.read_csv(observations)
observations_df = observations_df.round({'Lumped contaminant prediction': 4, 'Lumped contaminant observation': 4})
observations_df

,Lumped contaminant prediction,Lumped contaminant observation,Longitude,Latitude
0,0.0000,0.0000,12.516980,66.110426
1,0.0000,0.3962,25.067836,60.332961
2,2.4718,2.4683,24.865255,60.295969
3,0.0000,0.0000,25.046603,60.290225
4,2.4718,0.0000,24.962927,60.267419
...,...,...,...,...
245,0.2856,0.8228,23.575700,38.294717
246,0.4284,0.7203,23.494950,38.276783
247,0.0000,0.1029,32.895647,34.984621
248,0.0000,0.0854,32.889552,34.977839


In [8]:
#Read river graph
graph_location = os.path.join(data_directory, "river_graph.pkl")
open_graph = open(graph_location, "rb")
graph = pickle.load(open_graph)
open_graph.close()
ts = list(networkx.topological_sort(graph))

In [9]:
"""
A list of the parameters in the model, including excretion, attenuation, filtered efficacy,
                        primary efficacy, secondary efficacy and tertiary efficacy.
"""

parameters = [1.927644446155748, 0.005, 1, 0.33, 0.7, 0.92]

In [10]:
#Run simulation
g = graph_functions.run_model(graph, ts, cont_df, parameters)

In [11]:
#Remove nodes with concentration 0 (improves visualization)
remove = [node for node, data in g.nodes(data=True) if data["Contaminant"] == 0]
g.remove_nodes_from(remove)

In [12]:
#Select Nodes with only one antecessor and one predecessor
intermiediate_nodes = []    
for node in g.nodes():
    successors = list(g.successors(node))
    predecessors = list(g.predecessors(node))

    if len(predecessors) == 1 and len(successors) == 1:
        intermiediate_nodes.append(node)

#Remove some intermediate nodes
removal_factor = 1    #The more nodes you remove, the less sinuous the rivers will look
intermediate_to_remove = random.sample(intermiediate_nodes, math.floor(len(intermiediate_nodes) * removal_factor))
for node in intermediate_to_remove:
    successor = list(g.successors(node))[0]
    predecessor = list(g.predecessors(node))[0]
    g.remove_node(node)
    g.add_edge(predecessor, successor)

In [13]:
#Convert graph to dataframe

lat_src = []
lng_src = []
lat_dst = []
lng_dst = []
flow = []
relative_contaminant = []
label = []

for node in g.nodes(data=True): 
    src = node[0]
    src_data = node[1]
        
    successors = list(g.successors(src)) 
    if successors:
        dst = successors[0]
        dst_data = g.nodes[dst]
        lat_src.append(src_data["latitude"])
        lng_src.append(src_data["longitude"])
        lat_dst.append(dst_data["latitude"])
        lng_dst.append(dst_data["longitude"])
        flow.append(round(src_data["flow_HR"] / 3600, 4))
        relative_contaminant.append(round(src_data["Relative Contaminant"], 4))
        if src_data["Relative Contaminant"] < 1:
            label.append("0 - 1 ng/L")
        elif src_data["Relative Contaminant"] < 2:
            label.append("1 - 2 ng/L")
        elif src_data["Relative Contaminant"] < 4:
            label.append("2 - 4 ng/L")
        else:
            label.append("4+ ng/L")


df = pandas.DataFrame({
    "lat_src": lng_src,
    "lng_src": lat_src,
    "lat_dst": lng_dst,
    "lng_dst": lat_dst,
    "Flow (m3/s)": flow,
    "Concentration (ng/L)": relative_contaminant,
    "Concentration threshold": label
})

df

,lat_src,lng_src,lat_dst,lng_dst,Flow (m3/s),Concentration (ng/L),Concentration threshold
0,71.041667,25.554167,71.041667,25.545833,0.197,3.6513,2 - 4 ng/L
1,70.729167,27.616667,70.754167,27.758333,1.034,0.3052,0 - 1 ng/L
2,70.662500,23.687500,70.670833,23.670833,0.953,2.4220,2 - 4 ng/L
3,70.479167,26.329167,70.491667,26.550000,0.227,1.2094,1 - 2 ng/L
4,70.291667,29.783333,70.225000,30.475000,1.469,1.1425,1 - 2 ng/L
...,...,...,...,...,...,...,...
28135,34.895833,33.008333,34.891667,32.987500,0.148,2.5004,2 - 4 ng/L
28136,34.895833,33.591667,34.891667,33.629167,0.055,217.0226,4+ ng/L
28137,34.866667,32.866667,34.750000,32.920833,0.546,0.5420,0 - 1 ng/L
28138,34.750000,32.920833,34.650000,32.900000,2.357,0.5544,0 - 1 ng/L


In [14]:
#Read river basins file
river_basins_location = os.path.join(data_directory, "hydrobasins_europe/hydrobasins_europe.shp")   #Data from https://data.apps.fao.org/aquamaps/
river_basins_df = gpd.read_file(river_basins_location)
river_basins_df = river_basins_df[['MAJ_NAME', 'SUB_NAME', 'geometry']]

In [15]:
#For each node, get river basin name
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lng_src, df.lat_src), crs='EPSG:4326')

join_left_df = gdf.sjoin(river_basins_df, how="left")
df = join_left_df[['lat_src', 'lng_src', 'lat_dst', 'lng_dst', 'Flow (m3/s)', 'Concentration (ng/L)', 'MAJ_NAME', 'SUB_NAME', "Concentration threshold"]]
df = df.rename(columns={"MAJ_NAME": "FAO major basin", "SUB_NAME": "FAO minor basin"})
df = df.fillna('')
df

,lat_src,lng_src,lat_dst,lng_dst,Flow (m3/s),Concentration (ng/L),FAO major basin,FAO minor basin,Concentration threshold
0,71.041667,25.554167,71.041667,25.545833,0.197,3.6513,"Scandinavia, North Coast",Tenojoki,2 - 4 ng/L
1,70.729167,27.616667,70.754167,27.758333,1.034,0.3052,"Scandinavia, North Coast",Tenojoki,0 - 1 ng/L
2,70.662500,23.687500,70.670833,23.670833,0.953,2.4220,"Scandinavia, North Coast",Tenojoki,2 - 4 ng/L
3,70.479167,26.329167,70.491667,26.550000,0.227,1.2094,"Scandinavia, North Coast",Tenojoki,1 - 2 ng/L
4,70.291667,29.783333,70.225000,30.475000,1.469,1.1425,"Scandinavia, North Coast",Tenojoki,1 - 2 ng/L
...,...,...,...,...,...,...,...,...,...
28135,34.895833,33.008333,34.891667,32.987500,0.148,2.5004,,,2 - 4 ng/L
28136,34.895833,33.591667,34.891667,33.629167,0.055,217.0226,,,4+ ng/L
28137,34.866667,32.866667,34.750000,32.920833,0.546,0.5420,,,0 - 1 ng/L
28138,34.750000,32.920833,34.650000,32.900000,2.357,0.5544,,,0 - 1 ng/L


In [17]:
df.to_csv(os.path.join(results_directory, "river_segments.csv"))
cont_df_names.to_csv(os.path.join(results_directory, "discharge_points.csv"))
observations_df.to_csv(os.path.join(results_directory, "observations.csv"))